In [6]:
#importing the libraries
import pandas as pd 
import zipfile
import numpy
#unziping the files
def unzip(filename,folder_name):
    with zipfile.ZipFile(filename,'r') as zf:
        inner_file=zf.namelist()
        zf.extractall(folder_name)
        return inner_file
inner_file=unzip('sensor_data_2025.zip','Final_Assessment')
unzip('Final_Assessment/munich_data.zip','munich_data')
unzip('Final_Assessment/tokyo_data.zip','tokyo_data')
munich_csv=pd.read_csv('munich_data/readings.csv')
munich_csv['city']='munich'
tokyo_csv=pd.read_csv('tokyo_data/readings.csv')
tokyo_csv['city']='Tokyo'
munich_json=pd.read_json('munich_data/station_info.json')
munich_json['city']='munich'
tokyo_json=pd.read_json('tokyo_data/station_info.json')
tokyo_json['city']='tokyo'
all_reading_df=pd.concat([munich_csv,tokyo_csv])
all_station_info_df=pd.concat([munich_json,tokyo_json])

In [7]:
main_df=all_reading_df.merge(all_station_info_df,on='station_id')
main_df.head()

,station_id,timestamp,temperature_celsius,humidity_percent,pm2_5,city_x,location,status,deployment_year,city_y
0,1001,2025-02-04 23:00:00,14.17,77.11,22.80,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich
1,1001,2025-01-06 16:00:00,2.66,60.41,29.61,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich
2,1001,2025-01-01 13:00:00,0.31,53.84,42.90,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich
3,1001,2025-01-20 11:00:00,14.97,80.40,NaN,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich
4,1001,2025-02-12 06:00:00,14.22,78.14,62.48,munich,"{'latitude': 48.116274, 'longitude': 11.563626}",active,2024,munich


In [8]:
cleaned_df=main_df[main_df['status']=='active']
cleaned_df.dropna(subset=['temperature_celsius','pm2_5'],how='any',inplace=True)

In [11]:
cleaned_df_pm2=cleaned_df.groupby(['city_y'],as_index=False)['pm2_5'].mean()
cleand_df_temp=cleaned_df.groupby(['city_y'],as_index=False)['temperature_celsius'].mean()
result_df=cleand_df_temp.merge(cleaned_df_pm2,on=['city_y'])
result_df.head()

,city_y,temperature_celsius,pm2_5
0,munich,7.550826,40.038595
1,tokyo,12.685729,39.741027


In [13]:
resut=result_df.to_csv('city_summary_report_.csv',index=False)